#Regressione simbolica
Il presente codice esegue la regressione simbolica dei dati prodotti dalla rete neurale nel notebook GNN_Training.

Lo scopo è quello di trovare la funzione che meglio descrive la forza del sistema fisico preso in considerazione.

Come primo passo, viene installato Julia.

In [1]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.2"
JULIA_PACKAGES="PyCall SymbolicRegression"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"'
  done
  
  julia -e 'println("Success")'
fi

Output streaming troncato alle ultime 5000 righe.
  ◓ SplittablesBase
  Progress [==========================>              ]  69/110
  ◑ StaticArrays
  ◒ Tables
  ◑ SplittablesBase
  Progress [==========================>              ]  69/110
  ◒ StaticArrays
  ◐ Tables
  ◒ SplittablesBase
  Progress [==========================>              ]  69/110
  ◐ StaticArrays
  ◓ Tables
  ◐ SplittablesBase
  Progress [==========================>              ]  69/110
  ◓ StaticArrays
  ◑ Tables
  ◓ SplittablesBase
  Progress [==========================>              ]  69/110
  ◑ StaticArrays
  ◒ Tables
  ◑ SplittablesBase
  Progress [==========================>              ]  69/110
  ◒ StaticArrays
  ◐ Tables
  ◒ SplittablesBase
  Progress [==========================>              ]  69/110
  ◐ StaticArrays
  ◓ Tables
  ◐ SplittablesBase
  Progress [==========================>              ]  70/110
  ◓ StaticArrays
  ✓ Tables
  ◓ SplittablesBase
  ◑ LLVMExtra_jll
  Progress [============

In [2]:
#%pip install -Uq pysr pytorch_lightning

In [4]:
#from julia import Julia

#julia = Julia(compiled_modules=False)
#from julia import Main
#from julia.tools import redirect_output_streams

#redirect_output_streams()


Si fa poi l'accesso alla cartella di Google Drive Colab Notebook, dove è presente il file messages_over_time.pkl generato dalla GNN.

In [5]:
import pickle as pkl

In [7]:
from google.colab import drive
drive.mount("/content/drive")

%cd /content/drive/MyDrive/Colab Notebooks
#%cd /content/drive/MyDrive/GNN_project
%ls

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks
GN_Demo_Colab.ipynb  GNN_Training.ipynb  Symbolic_Regression.ipynb


Viene scaricato e caricato il file .pkl da utilizzare per la regressione.

In [8]:
file = open('messages_over_time.pkl', 'rb')

messages_over_time = pkl.load(file)

file.close()

FileNotFoundError: ignored

Viene visualizzato l'ultimo elemento dell'array di messaggi prodotti dal modello.

In [ ]:
messages_over_time[-1]

Viene selezionata la migliore componente dell'array di messaggi. Se nel training è stato usato un modello bottleneck, le componenti tra cui scegliere saranno due (pari alla dimensionalità del sistema fisico in questione).
Nel caso di regularizzation loss l1, la dimensionalità del messaggio è di 100 componenti.

In [ ]:
import numpy as np

best_message = np.argmax([np.std(messages_over_time[-1]['e%d'%(i,)]) for i in range(2)]) 

Viene quindi visualizzato il migliore messaggio.

In [ ]:
messages_over_time[-1][['e%d'%(best_message,), 'dx', 'dy', 'r', 'm1', 'm2']]

Si installa PySR.

In [ ]:

pip install pysr


Obiettivo della regressione simbolica: cercare una relazione tra le variabili "dx, dy, r, m1..." e la migliore componente del messaggio (ovvero della forza).
Le variabili indipendenti sono "dx, dy, r,...",
mentre la variabile dipendente è la componente migliore del messaggio.

In [ ]:
from pysr import PySRRegressor


In [ ]:
best = messages_over_time[-1][['e%d'%(best_message,), 'dx', 'dy', 'r', 'm1', 'm2']]

In [ ]:
X = best[['dx', 'dy', 'r', 'm1', 'm2']]
Y = best[['e%d'%(best_message,)]]

Qui si costruisce il modello per la regressione. In particolare:

1) model_selection seleziona il modello migliore in base ad una combinazione di semplicità e accuratezza;

2) viene dichiarato il numero di interazioni dell'algoritmo;

3) è possibile, selezionando "binary_operators" o "unary_operators", forzare il sistema a scegliere all'interno di un insieme di operatori dati;

4) "select_k_features = somenumber" è un comando che impone al modello di non usare tutte le variabili date (dx, dy, r, m1, m2, q1, q2), ma di scegliere le più significative per la regressione;

5) viene definita la funzione di loss che dovrà quantificare la bontà dei modelli sviluppati.




In [ ]:
model = PySRRegressor(
    model_selection="best",
    niterations=300,
     binary_operators=["+", "*","-","/"],
     #unary_operators=["cos", "exp", "sin", "inv(x) = 1/x"],
     unary_operators=["square", "cube"],
     #extra_sympy_mappings={"inv": lambda x: 1/x, "inv_q": lambda x: 1/x^2, "inv_cub": lambda x: 1/x^3},
     select_k_features=3,
    loss="L1DistLoss()",
    batching=True
)

model.fit(X, Y)

Viene stampato il modello, anche sotto forma di tabella in linguaggio LaTeX.

In [ ]:
print(model)

In [ ]:
print(model.latex_table())

In [ ]:
X[["r","m1"]].shape


Viene infine stampata la funzione con il miglior score.

In [ ]:
pysr_model.simpy()